In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    "udp_wcm_bronze_cx_loyalty",
    "lyt_loyalty_customers",
    proc_date,
    "temp_cx_loyalty_lyt_loyalty_customers"
)

In [0]:
spark.sql(f"""
    CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_silver_cx_loyalty.lyt_loyalty_customers
    TBLPROPERTIES (
        'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
        'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
    ) AS
    SELECT * FROM temp_cx_loyalty_lyt_loyalty_customers
    QUALIFY ROW_NUMBER() OVER (PARTITION BY CSN ORDER BY update_date DESC) = 1
""")